# Explorer les données d'OpenFoodFacts

Nous allons explorer le jeu de données d'OpenFoodFacts, filtré pour ne conserver que les produits vendus en France et les champs remplis pour au moins 1000 produits.

In [1]:
import pandas as pd

In [2]:
# modifier le chemin vers le fichier CSV si nécessaire
# ici on utilise le fichier filtré dans le notebook précédent
CSV_FILE = '../data/off_subset.csv'

In [3]:
# on charge les types de données qu'on a spécifiés dans le notebook 1
DTYPE_FILE = '../data/dtype.txt'
with open(DTYPE_FILE) as f:
    dtype = eval(f.read())
dtype

{'origins': 'category',
 'origins_tags': 'category',
 'first_packaging_code_geo': 'category',
 'cities_tags': 'category',
 'purchase_places': 'category',
 'countries': 'category',
 'countries_tags': 'category',
 'countries_en': 'category',
 'traces_tags': 'category',
 'traces_en': 'category',
 'additives': 'category',
 'ingredients_from_palm_oil_tags': 'category',
 'ingredients_that_may_be_from_palm_oil_tags': 'category',
 'nutriscore_grade': 'category',
 'pnns_groups_1': 'category',
 'pnns_groups_2': 'category',
 'states': 'category',
 'states_tags': 'category',
 'states_en': 'category',
 'cities': 'float16',
 'allergens_en': 'float16',
 'serving_quantity': 'float16',
 'no_nutriments': 'float16',
 'additives_n': 'float16',
 'ingredients_from_palm_oil_n': 'float16',
 'ingredients_from_palm_oil': 'float16',
 'ingredients_that_may_be_from_palm_oil_n': 'float16',
 'ingredients_that_may_be_from_palm_oil': 'float16',
 'nutriscore_score': 'float16',
 'nova_group': 'float16',
 'energy-kj_100g

In [4]:
df_off = pd.read_csv(CSV_FILE, sep='\t', dtype=dtype)
df_off.info(memory_usage='deep')

/home/mathieu/miniconda3/envs/off-explore/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643862 entries, 0 to 643861
Columns: 101 entries, code to nutrition-score-uk_100g
dtypes: category(18), float16(44), int64(2), object(37)
memory usage: 1.6 GB


## Exploration et nettoyage avec OpenRefine

La méthode la plus simple pour explorer le contenu du jeu de données est d'utiliser un outil graphique comme OpenRefine, qui permet aussi et surtout de nettoyer un jeu de données de façon reproductible.

### Présentation et installation d'OpenRefine

OpenRefine est un outil graphique, open source, pour travailler sur des jeux de données sales.
Cet outil permet de nettoyer un jeu de données de façon semi-automatique, par itérations successives.
OpenRefine stocke l'historique des opérations de nettoyage que vous appliquez et vous permet de l'exporter pour le rejouer, sur le même jeu de données ou sur une autre version de ce jeu de données.

Je vous conseille de visionner les 3 captures vidéo de la documentation officielle, qui donne une bonne idée de ce que vous pouvez réaliser avec OpenRefine.
https://github.com/OpenRefine/OpenRefine/wiki/Screencasts

Vous pouvez télécharger OpenRefine depuis cette page :
https://openrefine.org/download.html

### Lancement d'OpenRefine
Une fois que vous avez téléchargé et installé OpenRefine, vous pouvez ouvrir le fichier CSV filtré (tous les produits d'OpenFoodFacts distribués en France, restreint aux colonnes contenant plus de 1000 valeurs).

Par défaut, OpenRefine s'alloue 1 Go de RAM, ce qui est insuffisant pour notre fichier.
Il y a deux solutions:
1. Allouer plus de RAM pour ouvrir le fichier ;
2. Exporter en CSV un sous-ensemble plus restreint du jeu de données, contenant moins de lignes ou moins de colonnes.

#### Option 1: Allouer plus de RAM
On modifie les paramètres de lancement d'OpenRefine: https://github.com/OpenRefine/OpenRefine/wiki/FAQ:-Allocate-More-Memory

Par exemple :
```sh
# 4096 Mo = 4 Go de RAM
./refine -m 4096m
```

#### Option 2: Exporter un sous-ensemble du jeu de données
Nous pouvons exporter une fraction de l'ensemble de produits, par exemple 70000 lignes correspondent à un peu plus de 10% des produits.

In [6]:
# on prend les 70000 premières lignes
df_sub = df_off[:70000]
# on fait un dump CSV du jeu de données filtré
SEL_FILE = '../data/off_small.csv'
df_sub.to_csv(SEL_FILE, sep='\t', index=False)

### Utilisation d'OpenRefine

Ouvrir le fichier.

Dans la fenêtre de configuration des options de parsing:
- vérifier que le séparateur (tab), la ligne d'en-tête... sont correctement détectés ;
- cocher la case "Parse cell text into numbers, dates..." pour mieux visualiser et manipuler ces types de données ;
- appuyer sur "create project" (en haut à droite).

L'import des données ne prend normalement que quelques secondes.
Si le temps d'import estimé augmente et se compte en minutes, c'est que vous n'avez pas alloué suffisamment de RAM ou que vous essayez de charger un fichier trop gros (retour à l'option 1 ou 2 ci-dessus).

Nous pouvons créer une facette pour examiner une colonne, par exemple la colonne contenant les marques.

La facette met en évidence que parmi les plus de 60000 valeurs distinctes se trouvent de nombreuses variantes graphiques de la même marque.
L'utilisation d'une méthode de clustering dans une facette permet de regrouper et fusionner ces variantes par paquets.

Par exemple, combien y a-t-il de variantes pour la marque "Nos régions ont du talent, Leclerc" ?

Quelles sont les informations disponibles pour chaque cluster ?

Pour la méthode de clustering "key collision":
- Quelles "keying functions" sont disponibles?
- Comment fonctionnent-elles?
- Produisent-elles des clusterings similaires?
- Pourquoi?

## Exploration avec dirty_cat

```sh
pip install --user dirty_cat
```

https://dirty-cat.github.io/stable/auto_examples/01_investigating_dirty_categories.html

In [14]:
df_off['brands'].value_counts().sort_index()[200:230]

4 Pis                  1
4 x 109g               1
400 g                  1
45                     1
4TR                    1
4U                     1
4move                  1
5 Gum                  3
5 Oceans               3
5 Océans               1
50 ml                  1
550g                   1
555                    1
555,Palais impérial    1
5Gum                   3
6                      1
620g                   1
7 Days                 8
7 Days,Chipita         1
7 UP                   1
7 Up                   4
7 days                 2
7 up                   5
7 up, 7up, Seven Up    1
7 up,Seven-Up          1
7/e                    1
74cl                   1
75 g                   1
7D                     1
7Gün                   1
Name: brands, dtype: int64

In [17]:
sorted_values = df_off['brands'].sort_values().unique()

from dirty_cat import SimilarityEncoder

similarity_encoder = SimilarityEncoder(similarity='ngram')
transformed_values = similarity_encoder.fit_transform(
    sorted_values.reshape(-1, 1))

# TODO continuer

ModuleNotFoundError: No module named 'dirty_cat'

## Geopandas

Le champ `first_packaging_code_geo` contient des coordonnées géographiques.

Il est donc possible de dessiner une carte avec `geopandas`:
https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

In [5]:
# TODO dessiner une carte à partir de first_packaging_code_geo

## Exploration par facettes liées ?

https://seaborn.pydata.org/tutorial/relational.html#showing-multiple-relationships-with-facets

## Réduction de dimensionalité

Nous utiliserons une implémentation efficace de t-SNE: [openTSNE](https://github.com/pavlin-policar/openTSNE/).

```sh
conda install --channel conda-forge opentsne
```

In [ ]:
from openTSNE import TSNE

embedding = TSNE().fit(x)